## LLM Emotional Response For Paintings

### Model information: Gemini 2.5 flash
### output "free_emo_model.csv" 

In [15]:
#import libraries

from google import genai
from google.genai import types
from google.api_core import exceptions as google_exceptions
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import os
import re
import time
import random
import logging, sys
from pathlib import Path
import mimetypes

import pandas as pd
from tqdm import tqdm
from PIL import Image
from functools import reduce

logging.basicConfig(
    stream=sys.stdout,             
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S")

#### For each emotion, write one model to produce output. Plus one aesthetic rating model.

In [63]:
# ——— CONFIGURATION ———————————————————————————————————————————————————————————————————
IMAGE_FOLDER      = Path("/Users/Stella/Desktop/EmotionArt/emotion-art/data/exp_images/abstract")
OUTPUT_CSV        = Path("free_emo_model.csv")
MODEL_NAME        = "gemini-2.5-flash"
MAX_RETRIES       = 6
BACKOFF_BASE      = 3.0
BATCH_SIZE        = 10
MAX_OUTPUT_TOKENS = 2049
IMAGE_EXTS        = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"}

In [64]:
# ——— LOGGING ———————————————————————————————————————————————————————————————————————
logging.basicConfig(
    stream=sys.stdout,             
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S"
)

In [65]:
# ——— HELPER FUNCTIONS —————————————————————————————————————————————————————————————————
def get_mime_type(path: str) -> str:
    mime, _ = mimetypes.guess_type(path)
    return mime or "application/octet-stream"

def is_image_file(path: Path) -> bool:
    return path.is_file() and path.suffix.lower() in IMAGE_EXTS and not path.name.startswith(".")

_SCORE_RE = re.compile(r"^[A-Za-z]+:\s*([\d.]+)", re.MULTILINE)
_EXPL_RE  = re.compile(r"Explanation:\s*(.*)", re.DOTALL)

def parse_response(raw: str) -> tuple[float|None, str]:
    text = raw.decode("utf-8", errors="ignore") if isinstance(raw, (bytes, bytearray)) else str(raw or "")
    m_score = _SCORE_RE.search(text)
    m_expl  = _EXPL_RE.search(text)
    if not (m_score and m_expl):
        return None, text.strip()
    return float(m_score.group(1)), m_expl.group(1).strip()

#for batch processing
def chunker(seq, size):
    for i in range(0, len(seq), size):
        yield seq[i : i + size]


In [66]:
def free_emotion_prompt() -> str:
    return (
        "You are an average American citizen evaluating your emotional response to a painting.\n"
        "Provide a single free-form paragraph describing how viewing this painting makes you feel emotionally. "
        "Feel free to mention anything that helps convey your emotional reaction."
    )

In [67]:
def free_emotion_prompt2() -> str:
    return (
        "You are an average American citizen evaluating your emotional response to a painting.\n"
        "Provide a single free-form paragraph describing how viewing this painting makes you feel emotionally. "
        "Focus on all the emotional responses that come to mind, "
        "including any specific words or phrases that capture your emotional reaction.\n"
        "Only include what you feel, not what you do not feel or what it is not."
    )

In [68]:
# ——— PRELOAD IMAGES AND PROMPTS ——————————————————————————————————————————————————————————
all_images  = sorted(p for p in IMAGE_FOLDER.iterdir() if is_image_file(p))

# ——— VERTEX AI CLIENT FACTORY —————————————————————————————————————————————————————————
def make_vertex_client():
    return genai.Client(
        vertexai=True,
        project="emotion-art-analysis",
        location="global",
    )


In [69]:
# instead of .iterdir(), use .rglob() to find images in any subdirectory
all_images = sorted(
    p for p in IMAGE_FOLDER.rglob("*")
    if is_image_file(p)
)


In [70]:
# ——— CORE EMOTION CALL —————————————————————————————————————————————————————————————————
def emotion_model_vertex(image_path: str, client) -> dict:
    
    image_path = Path(image_path)
    image_name = image_path.name
    image_category = image_path.parent.name
    with open(image_path,"rb") as img_file:
        image_bytes = img_file.read()
        
    mime_type = get_mime_type(image_path)

    prompt_text = free_emotion_prompt2()

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.models.generate_content(
                model   = MODEL_NAME,
                contents = [
                    types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
                    types.Part.from_text(text=prompt_text),
                ],
                config = types.GenerateContentConfig(
                    max_output_tokens=MAX_OUTPUT_TOKENS
                )
            )

            raw = resp.candidates[0].content.parts[0].text if resp.candidates else ""
            
            return {
                "image": image_name,
                "image_category": image_category,
                "free_response":      raw
            }

        except google_exceptions.ServiceUnavailable:
            backoff = min(BACKOFF_BASE * 2 ** (attempt - 1), 30) + random.random()
            logging.warning(f"503 overload (try {attempt}), sleeping {backoff:.1f}s")
            time.sleep(backoff)

        except Exception as e:
            backoff = min(BACKOFF_BASE * 2 ** (attempt - 1), 30) + random.random()
            logging.warning(f"Error on try {attempt}: {e}")
            time.sleep(backoff)

    return {
        "image": image_name,
        "image_category": image_category,
        "free_response": raw
    }


In [71]:
def call_vertex_for_image(image_path: Path, client) -> dict:
    image_name     = image_path.name
    image_category = image_path.parent.name
    prompt_text    = free_emotion_prompt2()

    with open(image_path, "rb") as f:
        image_bytes = f.read()
    mime_type = get_mime_type(image_path)

    raw = ""
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.models.generate_content(
                model=MODEL_NAME,
                contents=[
                    types.Part.from_bytes(data=image_bytes, mime_type=mime_type),
                    types.Part.from_text(text=prompt_text),
                ],
                config=types.GenerateContentConfig(
                    max_output_tokens=MAX_OUTPUT_TOKENS
                )
            )
            raw = resp.candidates[0].content.parts[0].text if resp.candidates else ""
            break

        except google_exceptions.ServiceUnavailable:
            backoff = min(BACKOFF_BASE * 2**(attempt - 1), 30) + random.random()
            logging.warning(f"503 (try {attempt}), sleeping {backoff:.1f}s")
            time.sleep(backoff)

        except Exception as e:
            backoff = min(BACKOFF_BASE * 2**(attempt - 1), 30) + random.random()
            logging.warning(f"Error on try {attempt}: {e}", exc_info=True)
            time.sleep(backoff)

    return {
        "image":          image_name,
        "image_category": image_category,
        "free_response":  raw
    }

In [72]:
def process_all_images_in_batches(
    image_paths: list[Path],
    batch_size: int = BATCH_SIZE,
    max_workers: int = 8
) -> pd.DataFrame:
    client  = make_vertex_client()
    records = []

    # chunk through your images
    for batch in chunker(image_paths, batch_size):
        with ThreadPoolExecutor(max_workers=max_workers) as exe:
            futures = {
                exe.submit(call_vertex_for_image, img, client): img
                for img in batch
            }
            for future in as_completed(futures):
                img = futures[future]
                try:
                    rec = future.result()
                except Exception as e:
                    logging.error(f"❌ Error on {img.name}: {e}", exc_info=True)
                    rec = {
                        "image":          img.name,
                        "image_category": img.parent.name,
                        "free_response":  f"ERROR: {e}"
                    }
                records.append(rec)

    df = pd.DataFrame(records)
    return df

In [73]:
ouput_df = process_all_images_in_batches(all_images)
print("✅ All done:", ouput_df.shape)

17:51:06 [WARNING] Error on try 1: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}
Traceback (most recent call last):
  File "/var/folders/_k/84ry5gsj3bnd86w91nlcyck00000gr/T/ipykernel_88795/3697096895.py", line 13, in call_vertex_for_image
    resp = client.models.generate_content(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/google/genai/models.py", line 5898, in generate_content
    response = self._generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/google/genai/models.py", line 4838, in _generate_content
    response = self._api_client.request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/google/genai/_api_client.py", line 1067, in reques

In [75]:
ouput_df.to_csv('../output/free_emo_abstract.csv', index=False)

### Process images & save output

------END-------

### 882 images in total